# Algorithmic Fairness, Accountability, and Ethics, Spring 2023
# Exercise 3

## Task 0 (Setup)

We use the same dataset as in week 2. If you missed to install the module, please carry out the installation tasks at <https://github.com/zykls/folktables#basic-installation-instructions>.

After successful installation, you should be able to run the following code to generate a prediction task.
To make your life easier, we made the `BasicProblem`-magic from the `folktables` package (see exercises of week 2) explicit in this task.
This way, you can get access to different encodings of the data. 

**Note**: Some Windows users could not run the line `acs_data = data_source.get_data(states=["CA"], download=True)`. The dataset is available as a zip file on learnIT under week 2. The direct link is <https://learnit.itu.dk/mod/resource/view.php?id=174529>. Unzip it in the notebook's location, and set `download` to `False` in the code below.

In [111]:
from folktables.acs import adult_filter
from folktables import ACSDataSource
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # display all columns

from sklearn.model_selection import train_test_split
from IPython import display
from sklearn.preprocessing import StandardScaler, MinMaxScaler # TODO try using StandardScaler

In [15]:
def data_processing(data, features, target_name: str, threshold: float = 35000):
    df = data
    
    # Adult Filter (STARTS) (from Foltktables)
    df = df[~df["SEX"].isnull()]
    df = df[~df["RAC1P"].isnull()]
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    # Adult Filter (ENDS)

    # Groups of interest
    sex = df["SEX"].values
    
    # Target
    df["target"] = df[target_name] > threshold
    target = df["target"].values
    
    # we want to keep df before one_hot encoding to make Bias Analysis
    df = df[features + ["target", target_name]]
    
    df_processed = df[features].copy()
    cols = ["HINS1", "HINS2", "HINS4", "CIT",
            "COW", "SCHL", "MAR", "SEX", "RAC1P"]
    df_processed = pd.get_dummies(
        df_processed, prefix=None, prefix_sep='_', dummy_na=False, columns=cols, drop_first=True)
    df_processed = pd.get_dummies(
        df_processed, prefix=None, prefix_sep='_', dummy_na=True, columns=["ENG"], drop_first=True)
    
    return df_processed, df, target, sex

In [30]:
# fetch the data
data_source = ACSDataSource(
    survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)

feature_names = ['AGEP',  # Age
                 "CIT",  # Citizenship status
                 'COW',  # Class of worker
                 "ENG",  # Ability to speak English
                 'SCHL',  # Educational attainment
                 'MAR',  # Marital status
                 "HINS1",  # Insurance through a current or former employer or union
                 "HINS2",  # Insurance purchased directly from an insurance company
                 "HINS4",  # Medicaid
                 "RAC1P",  # Recoded detailed race code
                 'SEX']  # 1 -> male, 2 -> female

# preprocess the data
data, data_original, target, sex = data_processing(data=acs_data,
                                                   features=feature_names,
                                                   target="PINCP",  # total persons income
                                                   threshold=35000)

# split into train and test
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    data, target, sex, test_size=0.2, random_state=0)

In [125]:
np.unique(group_train)

array([1, 2])

# Task 1 (Logistic regression - Classification)

1) Train a logistic regression classifier on the training dataset. In our setup, the following parameters worked out well: `LogisticRegression(max_iter=5000, penalty = "l2", C= 0.8497534359086438, tol=1e-4, solver = "saga")`. Which scaling considerations do you think are necessary?
2) Report on the accuracy of the model. (If you are interested: How is the classification accuracy on the original dataset with categorial input?)
3) Report on the model weights (sort them by weight). Which weights are most important? Explain the influence of the most important weights. (For example, "being female instead of male increases/decreases the odds for ... by ...".)
4) Find a negative or a positive instance, and discuss how you can use the weights discussion to create a counterfactual. (E.g., "By increasing/decreasing feature ... to ..., the person is classified as ...").

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=5000, 
                           penalty="l2", 
                           C=0.8497534359086438, 
                           tol=1e-4, 
                           solver="saga")

In [109]:
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler() 

# Using StandardScaler
X_train['AGEP'] = ss_scaler.fit_transform(np.array(X_train['AGEP']).reshape(-1, 1))
X_test['AGEP'] = ss_scaler.fit_transform(np.array(X_test['AGEP']).reshape(-1, 1))

display(X_train.head())

# Using MinMaxScaler
# X_train['AGEP'] = mm_scaler.fit_transform(np.array(X_train['AGEP']).reshape(-1, 1))
# X_test['AGEP'] = mm_scaler.fit_transform(np.array(X_test['AGEP']).reshape(-1, 1))

model.fit(X_train, y_train)
model.score(X_test, y_test)

,AGEP,HINS1_2,HINS2_2,HINS4_2,CIT_2,CIT_3,CIT_4,CIT_5,COW_2.0,COW_3.0,COW_4.0,COW_5.0,COW_6.0,COW_7.0,COW_8.0,SCHL_2.0,SCHL_3.0,SCHL_4.0,SCHL_5.0,SCHL_6.0,SCHL_7.0,SCHL_8.0,SCHL_9.0,SCHL_10.0,SCHL_11.0,SCHL_12.0,SCHL_13.0,SCHL_14.0,SCHL_15.0,SCHL_16.0,SCHL_17.0,SCHL_18.0,SCHL_19.0,SCHL_20.0,SCHL_21.0,SCHL_22.0,SCHL_23.0,SCHL_24.0,MAR_2,MAR_3,MAR_4,MAR_5,SEX_2,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9,ENG_2.0,ENG_3.0,ENG_4.0,ENG_nan
358945,0.218604,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
275788,0.151449,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
141517,-0.184328,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
66729,1.091623,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
268579,-1.325968,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1


0.7696062147037027

The accuracy score are exactly the same despite using no scaler on AGEP, using MinMaxScaler, and StandardScaler

In [128]:
weights_unstorted = dict(zip(model.coef_[0], X_train.columns))
weights = dict(sorted(weights_unstorted.items())) # sort the weights to see which features have higher weights
weights

{-1.2166513058445103: 'COW_8.0',
 -1.016937167772307: 'HINS1_2',
 -0.9821688412772476: 'ENG_4.0',
 -0.8190440534406135: 'SEX_2',
 -0.7127802372194527: 'MAR_5',
 -0.6478483877880707: 'ENG_3.0',
 -0.6209991212399504: 'COW_6.0',
 -0.4725452088165363: 'SCHL_4.0',
 -0.2913311785179963: 'ENG_2.0',
 -0.21769210153764357: 'RAC1P_2',
 -0.21395253942625703: 'MAR_4',
 -0.2055665766732701: 'RAC1P_5',
 -0.1988020263236385: 'MAR_2',
 -0.17220615272337786: 'RAC1P_3',
 -0.14975231585091506: 'CIT_5',
 -0.14892775799892993: 'SCHL_5.0',
 -0.14694513870550605: 'SCHL_8.0',
 -0.11716321035943024: 'CIT_2',
 -0.11548795994830634: 'HINS2_2',
 -0.10465240601235644: 'RAC1P_9',
 -0.10075826398128335: 'SCHL_14.0',
 -0.05181173146373686: 'RAC1P_7',
 -0.05157412535562554: 'RAC1P_8',
 -0.043662904375582234: 'SCHL_6.0',
 -0.03754770365627234: 'RAC1P_4',
 -0.03623947615614551: 'RAC1P_6',
 -0.03578011900316776: 'COW_2.0',
 0.006281617465736105: 'CIT_3',
 0.033098706376258794: 'MAR_3',
 0.04977576197531114: 'COW_3.0',
 0

In [133]:
dict(zip(np.exp(model.coef_[0]), X_train.columns))

{1.7455488416026406: 'AGEP',
 0.3617010750650657: 'HINS1_2',
 0.8909312990313982: 'HINS2_2',
 2.305294302063562: 'HINS4_2',
 0.8894400154918636: 'CIT_2',
 1.0063013882004472: 'CIT_3',
 1.0697448743203843: 'CIT_4',
 0.8609211865511036: 'CIT_5',
 0.9648524228731948: 'COW_2.0',
 1.0510353878503549: 'COW_3.0',
 1.0728575136697498: 'COW_4.0',
 1.8373611907860607: 'COW_5.0',
 0.537407234291464: 'COW_6.0',
 1.306695127059448: 'COW_7.0',
 0.296220459626802: 'COW_8.0',
 1.5730306886925238: 'SCHL_2.0',
 1.74814751216161: 'SCHL_3.0',
 0.6234135296925115: 'SCHL_4.0',
 0.8616313586241573: 'SCHL_5.0',
 0.9572765968419846: 'SCHL_6.0',
 1.0734717470848252: 'SCHL_7.0',
 0.8633413401406276: 'SCHL_8.0',
 1.0548052920708093: 'SCHL_9.0',
 1.197097435693877: 'SCHL_10.0',
 1.1179302741088548: 'SCHL_11.0',
 1.2795977439985375: 'SCHL_12.0',
 1.1322242426314137: 'SCHL_13.0',
 0.9041515724718009: 'SCHL_14.0',
 1.2323964576693243: 'SCHL_15.0',
 1.5584552278167398: 'SCHL_16.0',
 1.8516536769294467: 'SCHL_17.0',
 1

In [147]:
predictions = model.predict(X_test)

X_test['prediction'] = predictions

In [159]:
## ANSWER
len(X_test.loc[X_test["SCHL_24.0"] == 1]) / len(X_test.loc[X_test["SCHL_24.0"] == 0])

0.020497040185672933

In [137]:
# calculating the odds ratio for each prediction
for i in model.predict_proba(X_test):
    print(i[1] / i[0]) # odds ratio -> p(y=1) / p(y=0)

6.2677541980203655
8.149719514084184
1.5130780726370365
1.30440887131289
5.5788171871025485
0.2796864544597421
0.6334339901779276
0.08655047529175483
0.141229672212681
0.9041271581626159
11.719261504487175
9.420276893670305
0.6579383742841446
0.5488739895264819
0.5136643732255973
13.174769570902066
1.0532875383198292
2.2634897668442173
0.6506040203342657
18.62475704501559
15.525931790256648
7.1698649405779635
1.9504020785137086
2.185028645473257
2.854588202274432
1.9539012504339124
0.0721109945526275
1.0424728777606016
43.850151155639615
0.6956069572749511
0.30560546806054695
2.9254463465213885
6.024623991852406
0.6376102454224926
0.8400222606460833
0.6642867689658484
0.9355632267853182
0.12406997763288466
0.34168889095893495
4.403224439569899
2.1898898111031
4.864662020257654
1.7848616562708024
0.7757318090311824
0.5413197622666405
1.329794059079145
0.19512222757812298
2.689625485976159
16.53044335697553
3.990756133935122
0.0455029134699227
0.03296353484193353
4.213291376774797
2.2416

# Task 2 (Decision tree)

1. Train a decision tree classifier on the training dataset. (You can work on the original dataset or on the one-hot encoded one.) The following parameter choices worked well in our setup: `(DecisionTreeClassifier(min_samples_split = 0.01, min_samples_leaf= 0.01, max_features="auto", max_depth = 15, criterion = "gini", random_state = 0))` Report on its accuracy. Visualize the tree using `plot_tree` from `sklearn`. Which parameters can you change the adapt the size of the tree? Try to find parameters that make the tree easier to understand.
2. For two training examples, explain their classification given the decision tree.
3. Compute feature importance as shown in the lecture. Which features are most important?
4. Provide a counterfactual, as in Task 2.


# Task 3 (Comparison)

Now you have both an interpretable logistic regression model and an interpretable decision tree model. Reflect on the explanations you can obtain from these two models: Do explanations from one model translate to the other? Do the counterfactuals from one work in the other model?  